# Sommelier Agent Demo

To simplify the content of the demo, this has been split into "setup" and "demo" notebooks. This is the Demo notebook, and assumes the Setup notebook has been run and validated, and that the Astra database has been populated with the demo data.

## Load `.env`

In [9]:
import os
from dotenv import load_dotenv
if not load_dotenv('.env',override=True):
    raise Exception("Couldn't load .env file")

## Wine Pairing Vector (ANN) Search
Using Langchain, we can use an engineered prompt to search for wines that pair well with a given list of foods. Here we are are relying on the general LLM model to provide the context for the search, with no specific knowledge of wines in our database.

In [10]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'], temperature=0.0)

# Generate wine pairing suggestions based on the user's input
food_list = "salmon, broccoli, rice, and hollandaise sauce"

prompt = f"What are some suitable wine types, flavors, and aroma profiles for a meal with {food_list}? Please respond in format that would be suitable for searching a database of professional wine reviews."
flavor_response = llm(prompt)
print(flavor_response)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




Type: White Wine
Flavors: Citrus, Stone Fruit, Herbal
Aroma Profiles: Lemon, Peach, Thyme, Oak


## Establish Connection to Astra
In preparation for using Astra as a vector store, we need to establish a connection to the database.

In [11]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {'secure_connect_bundle': os.environ['ASTRA_SECUREBUNDLE_PATH']}
auth_provider = PlainTextAuthProvider(os.environ['ASTRA_CLIENT_ID'], os.environ['ASTRA_CLIENT_SECRET'])
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

## Define a Langchain VectorStore
We can use the Astra connection to define a Langchain VectorStore.

In [12]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Cassandra

# This is the model we used to generate our embeddings
embed_model = "text-embedding-ada-002"

embedding_function = OpenAIEmbeddings(model=embed_model)

cassVstore = Cassandra(
    embedding=embedding_function,
    session=session,
    keyspace=os.environ['ASTRA_KEYSPACE'],
    table_name=os.environ['ASTRA_TABLE']
    )

## Search the VectorStore for Wine Pairings
Our `flavor_response` above was requested to be in a format sutable for searching a database of professional wine reviews. Let's use that to search for wines that pair well with our meal.

In [13]:
wine_matches = cassVstore.similarity_search(flavor_response, k=3)

for i, d in enumerate(wine_matches):
    print(f"\n## Wine {i}\n")
    print(d.page_content)
    print(d.metadata)


## Wine 0

Bright grapefruit peel and star fruit notes are laced with bouncy thyme and sweet pea flavors. Drink now. 9,000 cases made, 700 cases imported.
{'Review Date': '2021-03-29T00:00:00', 'Category': 'Less than $20', 'Score': 86, 'Winery': 'CHÂTEAU LAMOTHE VINCENT', 'Winery URL': '/wine/wine-detail/id/1243131/name/bordeaux-white-2018', 'Wine Name': 'Bordeaux White 2018', 'Wine URL': '/wine/wine-detail/id/1243131/name/bordeaux-white-2018', 'Price': 12.0, 'Reviewer': 'James Molesworth'}

## Wine 1

This has a plush feel, with a light shortbread note around the white peach, green fig and lime flavors. Verbena and honeysuckle accents add bounce on the finish. Vermentino. Drink now. 10,000 cases made, 2,000 cases imported.
{'Review Date': '2018-11-07T00:00:00', 'Category': '$20 to $40', 'Score': 89, 'Winery': 'DOMAINE PETRONI', 'Winery URL': '/wine/wine-detail/id/455320/name/corse-white-2017', 'Wine Name': 'Corse White 2017', 'Wine URL': '/wine/wine-detail/id/455320/name/corse-white-

These results are the reviews from our Wine Spectator database that have flavors that would be best-paried with our food list. These are as a wine expert would describe them, but a sommelier would tell us why they pair well.

## Sommelier Prompt
Let's use a prompt to generate a sommelier-style description of each wine.

In [14]:
prompts=[]
for i, d in enumerate(wine_matches):
    pairing_prompt = f"Given a meal with {food_list}, why would you pair it with {d.metadata['Wine Name']}, described as {d.page_content}? The response should be as a sommelier would describe it."
    prompts.append(pairing_prompt)

wines = llm.generate(prompts)

for i, generation in enumerate(wines.generations):
    description = generation[0].text.strip('\n')
    print(f"Wine {i}: {description}\n")

Wine 0: This Bordeaux White 2018 is a great pairing for this meal. Its bright grapefruit peel and star fruit notes will bring out the sweetness of the salmon, while the thyme and sweet pea flavors will complement the earthy notes of the broccoli. The acidity of the wine will also help to cut through the richness of the hollandaise sauce, making for a balanced and enjoyable meal.

Wine 1: This Corse White 2017 is a great pairing for this meal. Its plush feel and light shortbread note will complement the salmon and hollandaise sauce, while its white peach, green fig, and lime flavors will bring out the sweetness of the broccoli. The verbena and honeysuckle accents will add a nice brightness to the rice, and the finish will be refreshing and light. This is a great choice for a summer meal.

Wine 2: This Val De Loire Vignes Blanches 2017 is a great pairing for this meal. The thyme, lime, and oyster shell notes in the wine will bring out the delicate flavors of the salmon, while the crackli

This is more like it! Whether or not this makes any _real_ sense would take somebody who knows something about wine, but the principle certainly seems to work!